In [1]:
import pickle
from utils.analysis_from_interaction import *
from egg.core.language_analysis import Disent
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim

# calculate metrics from stored interactions

In [6]:
datasets = ('(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)')
n_attributes = (3, 3, 3, 4, 4, 5)
n_values = (4, 8, 16, 4, 8, 4)
n_epochs = 300
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [3]:
datasets = ('(4,4)',)
n_attributes = (4,)
n_values = (4,)
n_epochs = 100
#paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]

In [7]:
context_unaware = False # whether original or context_unaware simulations are evaluated
length_cost = False # whether length_cost was applied
no_cost = True # set to True to consider baselines without a cost
if length_cost:
    if context_unaware and not no_cost:
        setting = 'length_cost/context_unaware'
    elif context_unaware and no_cost:
        setting = 'length_cost/no_cost_context_unaware'
    elif not context_unaware and not no_cost:
        setting = 'length_cost/context_aware'
    else:
        setting = 'length_cost/no_cost_context_aware'
else:
    if context_unaware:
        setting = 'context_unaware'
    else:
        setting = 'standard'

### entropy scores: MI, effectiveness, efficiency

In [8]:
for d in range(len(datasets)):
    
    for run in range(5):

        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/' 
        print(path_to_run)
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = information_scores(interaction, attributes, values, normalizer="arithmetic")

        pickle.dump(scores, open(path_to_run + 'entropy_scores.pkl', 'wb'))

results/(3,4)_game_size_10_vsf_3/standard/0/


FileNotFoundError: [Errno 2] No such file or directory: 'results/(3,4)_game_size_10_vsf_3/standard/0/interactions/train/epoch_300/interaction_gpu0'

###  message length

In [ ]:
# we evaluated message length per hierarchy level after training but 
# you can also use the HierarchicalMessageLength callback and store the results 
# TODO: Message length results look weird, needs to be fixed!

for d in range(len(datasets)):
    
    for run in range(5): 
        
        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = message_length_per_hierarchy_level(interaction, attributes)
        
        pickle.dump(scores, open(path_to_run + 'message_length_hierarchical.pkl', 'wb'))

tensor([[ 3,  2,  3,  0],
        [ 2, 13,  3,  0],
        [ 1, 11, 11,  0],
        ...,
        [ 2,  7,  6,  0],
        [ 6, 11, 11,  0],
        [ 2, 13,  2,  0]])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1

###  symbol redundancy

In [ ]:
for d in range(len(datasets)):
    
    attributes = n_attributes[d]
    values = n_values[d]
    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    for run in range(5): 
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        #symbol_f = np.load(path_to_run + 'symbols_pernsame.npy')
        #path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)
        redundancy, MI = symbol_frequency(interaction, attributes, values, vocab_size)
        
        scores = {'symbol_redundancy': redundancy, 'MI_symbol-attribute_value': MI}
        
        pickle.dump(scores, open(path_to_run + 'symbol_redundancy.pkl', 'wb'))

c:\Users\fjapt\OneDrive\Desktop\reps\emergent-abstractions\utils\analysis_from_interaction.py:393: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
c:\Users\fjapt\OneDrive\Desktop\reps\emergent-abstractions\utils\analysis_from_interaction.py:393: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
c:\Users\fjapt\OneDrive\Desktop\reps\emergent-abstractions\utils\analysis_from_interaction.py:393: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
c:\Users\fjapt\OneDrive\Desktop\reps\emergent-abstractions\utils\analysis_from_interaction.py:393: RuntimeWarning: invalid value encountered in divide
  return symbol_frequency / att_val_frequency, mutual_information
c:\Users\fjapt\OneDrive\Desktop\reps\emergent-abstractions\utils\analysis_from_interaction.py:393: RuntimeWarning: invalid value enc

UnboundLocalError: cannot access local variable 'max_symbol' where it is not associated with a value

###  compositionality scores: topsim, posdis, bosdis

#### topsim

In [ ]:
# topsim
# although topsim values are stored throughout training if callbacks are verbose, we reevaluate the
# final topsim scores with more data points 

samples = 5000 # maybe shuffle from these because otherwise I just take the first 5,000 (which might not be the best)
for d, dataset in enumerate(datasets):
    
    dim = [n_values[d]]*n_attributes[d]
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        topsim_final = {}
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        for mode in ['train', 'val']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'val':
                interaction = torch.load(path_to_interaction_val)
                
                  
            messages = interaction.message.argmax(dim=-1)
            sender_input = interaction.sender_input
            n_targets = int(sender_input.shape[1]/2)
            # get target objects and fixed vectors to re-construct concepts
            target_objects = sender_input[:, :n_targets]
            target_objects = k_hot_to_attributes(target_objects, n_values[d])
            # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
            (objects, fixed) = retrieve_concepts_sampling(target_objects)
            # add one such that zero becomes an empty attribute for the calculation (_)
            objects = objects + 1
            concepts = torch.from_numpy(objects * (np.array(fixed)))
            specific_idx = np.where(np.sum(fixed, axis=1)==n_attributes[d])[0]
            messages_specific = messages[specific_idx]
            concepts_specific = concepts[specific_idx]
            
            generic_idx = np.where(np.sum(fixed, axis=1)==1)[0]
            messages_generic = messages[generic_idx]
            concepts_generic = concepts[generic_idx]

            messages = [msg.tolist() for msg in messages]
            messages_specific = [msg.tolist() for msg in messages_specific]
            messages_generic = [msg.tolist() for msg in messages_generic]

            encoded_input = encode_target_concepts_for_topsim(sender_input)
            # randomly take samples when more than 5000 samples are available
            # if len(encoded_input) > samples: 
            #     print("sampling")
            #     sample_indices = random.sample(range(len(encoded_input)), samples)
            #     sampled_input = [encoded_input[i] for i in sample_indices]
            #     sampled_messages = [messages[i] for i in sample_indices]
            #     print("start computing")
            #     print(len(sampled_input), len(sampled_input[0]), len(sampled_input[0][0]))
            #     topsim = TOPSIM.compute_topsim(sampled_input, sampled_messages)
            # else:
            topsim = TOPSIM.compute_topsim(encoded_input[0:samples], messages[0:samples]) # default: hausdorff distance for concepts, edit distance for messages
            # if len(concepts_specific) > samples:
            #     print("sampling specific")
            #     sample_indices_specific = random.sample(range(len(concepts_specific)), samples)
            #     sampled_input_specific = [concepts_specific[i] for i in sample_indices_specific]
            #     sampled_messages_specific = [messages_specific[i] for i in sample_indices_specific]
            #     topsim_specific = TOPSIM.compute_topsim(sampled_input_specific, sampled_messages_specific, 
            #                                             meaning_distance_fn="edit")
            # else:
            topsim_specific = TOPSIM.compute_topsim(concepts_specific[0:samples], messages_specific[0:samples], 
                                                        meaning_distance_fn="edit")
            
            topsim_generic = TOPSIM.compute_topsim(concepts_generic[0:samples], messages_generic[0:samples],
                                                   meaning_distance_fn="edit")

            print('... topsim computed')

            topsim_final['topsim_' + mode] = topsim
            topsim_final['topsim_specific_' + mode] = topsim_specific
            topsim_final['topsim_generic_' + mode] = topsim_generic
    
        pickle.dump(topsim_final, open(path_to_run +  "topsim_final.pkl", "wb" ) )
        print(topsim_final)  

dataset (3,4) run 0
... topsim computed
... topsim computed
{'topsim_train': 0.3860787629694438, 'topsim_specific_train': 0.45572987541232396, 'topsim_generic_train': 0.24778142416089613, 'topsim_val': 0.3950412254020727, 'topsim_specific_val': 0.48626573800026324, 'topsim_generic_val': 0.21846282372598158}
dataset (3,4) run 1
... topsim computed
... topsim computed
{'topsim_train': 0.4125152917925917, 'topsim_specific_train': 0.4904470244460408, 'topsim_generic_train': 0.34982507675734664, 'topsim_val': 0.42021641545540445, 'topsim_specific_val': 0.5030676764929992, 'topsim_generic_val': 0.4699486791514816}
dataset (3,4) run 2
... topsim computed
... topsim computed
{'topsim_train': 0.43035170469888995, 'topsim_specific_train': 0.4809216473945796, 'topsim_generic_train': 0.30507801048760247, 'topsim_val': 0.39774004771065247, 'topsim_specific_val': 0.4821887425900421, 'topsim_generic_val': 0.10150844357213834}
dataset (3,4) run 3
... topsim computed
... topsim computed
{'topsim_train'

ValueError: A 2-dimensional array must be passed.

#### Topsim over time

In [ ]:
for d, dataset in enumerate(datasets):
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        for mode in ['train', 'val']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'val':
                interaction = torch.load(path_to_interaction_val)

        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        messages = [msg.tolist() for msg in messages]
        encoded_input = encode_target_concepts_for_topsim(sender_input)
        dim = [n_values[0]] * n_attributes[0]
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        samples = 5000
        num_batches = len(messages) // samples + (len(messages) % samples > 0)
        topsim_over_time = []
        
        for i in range(num_batches):
            messages_batch = messages[i * samples:(i + 1) * samples]
            topsim = TOPSIM.compute_topsim(encoded_input[i * samples:(i + 1) * samples], messages_batch)
            topsim_over_time.append(topsim)
            
        pickle.dump(topsim_over_time, open(path_to_run +  "topsim_over_time.pkl", "wb" ) )

data set [4, 4, 4]
{'posdis_specific': 0.08637324720621109, 'bosdis_specific': 0.24354112148284912, 'posdis_generic': 0.051591720432043076, 'bosdis_generic': 0.18509045243263245, 'posdis': 0.059153079986572266, 'bosdis': 0.20398299396038055}
{'posdis_specific': 0.1363789588212967, 'bosdis_specific': 0.20830155909061432, 'posdis_generic': 0.057058822363615036, 'bosdis_generic': 0.15597590804100037, 'posdis': 0.1321350485086441, 'bosdis': 0.20890992879867554}
{'posdis_specific': 0.09997209161520004, 'bosdis_specific': 0.16721627116203308, 'posdis_generic': 0.10703134536743164, 'bosdis_generic': 0.17613279819488525, 'posdis': 0.08310873061418533, 'bosdis': 0.1597554236650467}
{'posdis_specific': 0.10748641937971115, 'bosdis_specific': 0.2282559871673584, 'posdis_generic': 0.10946443676948547, 'bosdis_generic': 0.10632359236478806, 'posdis': 0.07508981972932816, 'bosdis': 0.1828385442495346}
{'posdis_specific': 0.12036856263875961, 'bosdis_specific': 0.1979639083147049, 'posdis_generic': 0

#### Posdis and Bosdis

In [ ]:
# use Disent callback from egg

for d in range(len(datasets)): 
    
    path = paths[d]
    dim = [n_values[d]] * n_attributes[d]
    n_features = n_attributes[d] * n_values[d]
    vs_factor = int(path[-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    print("data set", dim)
    
    for run in range(5):
        
        posdis_bosdis = {}
    
        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        
        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[d])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects)
        # add one such that zero becomes an empty attribute for the calculation (_)
        objects = objects + 1
        concepts = torch.from_numpy(objects * (np.array(fixed)))

        # concrete/specific concepts: where all attributes are fixed
        concepts_specific = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]])
        messages_specific = messages[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]]

        # generic concepts: where only one attribute is fixed
        concepts_generic = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == 1])
        messages_generic = messages[torch.sum(torch.from_numpy(fixed), dim=1) == 1]
        
        posdis_specific = Disent.posdis(concepts_specific, messages_specific)
        bosdis_specific = Disent.bosdis(concepts_specific, messages_specific, vocab_size)

        posdis_generic = Disent.posdis(concepts_generic, messages_generic)
        bosdis_generic = Disent.bosdis(concepts_generic, messages_generic, vocab_size)
        
        posdis = Disent.posdis(torch.from_numpy(objects), messages)
        bosdis = Disent.bosdis(torch.from_numpy(objects), messages, vocab_size)
        
        posdis_bosdis['posdis_specific'] = posdis_specific
        posdis_bosdis['bosdis_specific'] = bosdis_specific
        posdis_bosdis['posdis_generic'] = posdis_generic
        posdis_bosdis['bosdis_generic'] = bosdis_generic
        posdis_bosdis['posdis'] = posdis
        posdis_bosdis['bosdis'] = bosdis

        print(posdis_bosdis)
    
        pickle.dump(posdis_bosdis, open(path_to_run + "posdis_bosdis.pkl", "wb" ) )

TypeError: 'float' object is not callable

#### Posdis and bosdis concept x context

In [ ]:
# bosdis concept x context
for d in range(len(datasets)):

    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    for run in range(5):

        path_to_run = paths[d] + '/' + str(setting) +'/' + str(run) + '/' 
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = bosdis(interaction, attributes, values, vocab_size)

        pickle.dump(scores, open(path_to_run + 'bosdis_scores.pkl', 'wb'))
# posdis concept x context
for d in range(len(datasets)):

    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1

    for run in range(5):
        path_to_run = paths[d] + '/' + str(setting) + '/' + str(run) + '/'
        print(path_to_run)
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        #print(path_to_interaction)
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = posdis(interaction, attributes, values, vocab_size)

        pickle.dump(scores, open(path_to_run + 'posdis_scores.pkl', 'wb'))

### co-occurrences

In [ ]:
# Not yet implemented:

for d in range(len(datasets)):
    
    vs_factor = int(paths[d][-2])
    
    for run in range(5): 
        
        path_to_run = paths[d] + str(setting) +'/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        
        scores = cooccurrence_per_hierarchy_level(interaction, attributes, values, vs_factor)

        print(scores)
        
        pickle.dump(scores, open(path_to_run + 'normalized_cooccurrence.pkl', 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: 'results/(3,4)_game_size_4_vsf_3standard/0/interactions/train/epoch_300/interaction_gpu0'